In [1]:
!pip install opencv-python

In [1]:
import cv2
import numpy as np

In [30]:
img1 = cv2.imread("images.jpeg")

cv2.imshow("XYZ1.jpeg", img1)
cv2.waitKey(0)

-1

In [31]:
def preprocess(img):
    
    img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    img_gaussian = cv2.GaussianBlur(img_grayscale, (5,5), 1)
    
#     print(img_gaussian.shape)

    img_threshold = cv2.adaptiveThreshold(img_gaussian, 255, 1, 1, 11, 2)
    
    return img_threshold

In [32]:
cv2.imshow("XYZ2.jpeg", preprocess(img1))
cv2.waitKey(0)

-1

In [33]:
def draw_contour(img_original, img_threshold):
    
    contours, hierarchy = cv2.findContours(img_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours, cv2.drawContours(img_original, contours, -1, (0, 255, 255), 3)

In [34]:
img_threshold = preprocess(img1)

contours, img_contour = draw_contour(img1, img_threshold)

cv2.imshow("XYZ3.jpeg", img_contour)
cv2.waitKey(0)

-1

In [42]:
def biggest_contour(contours):
    
    biggest = np.array([])
    max_area = 0
    
    for i in contours:
        area = cv2.contourArea(i)
        
        if area > 50:
            peri = cv2.arcLength(i, True)
            
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            
            if area > max_area and len(approx) == 4:
                biggest = approx
                
                max_area = area
                
    return biggest, max_area

In [43]:
biggest, max_area = biggest_contour(contours)

print(biggest, max_area)

[[[  0 118]]

 [[  0 121]]

 [[  3 121]]

 [[  3 118]]] 9.0


In [44]:
def reorder(points):
    points = points.reshape((4, 2))
    new_points = np.zeros((4, 1, 2), dtype=np.int32)
    
    add = points.sum(1)
    new_points[0] = points[np.argmin(add)]
    new_points[3] = points[np.argmax(add)]
    
    diff = np.diff(points, axis=1)
    new_points[1] = points[np.argmin(diff)]
    new_points[2] = points[np.argmax(diff)]
 
    return new_points

In [45]:
print(biggest, reorder(biggest))

[[[  0 118]]

 [[  0 121]]

 [[  3 121]]

 [[  3 118]]] [[[  0 118]]

 [[  3 118]]

 [[  0 121]]

 [[  3 121]]]


In [46]:
biggest = reorder(biggest)

cv2.drawContours(img_contour, biggest, -1, (0, 0), 25)

pts1 = np.float32(biggest)
pts2 = np.float32([[0, 0], [450, 0], [0, 450], [450, 450]])

matrix = cv2.getPerspectiveTransform(pts1, pts2)

img_wrap_colored = cv2.warpPerspective(img1, matrix, (450, 450))

img_wrap_colored = cv2.cvtColor(img_wrap_colored, cv2.COLOR_BGR2GRAY)

cv2.imshow("XYZ4.jpeg", img_wrap_colored)
cv2.waitKey(0)

-1